# アユムのファイル

In [1]:
!pip install ultralytics

In [ ]:
import numpy as np
from ultralytics import YOLO
import cv2
import os


# --- モデルの準備 ---
model = YOLO("yolo11n-pose.pt") # 学習済みのモデルファイル

# パラメータの定義
# 入力例: 画像ファイル, 動画ファイル, フォルダ ("images/"), またはカメラ(0)
source = "/2011tokyo_mister_fp-011-320x480.jpg" # 入力ソース
imgsz = 640 # 入力画像サイズ
conf = 0.25 # 信頼度閾値
save_dir = "runs/pose_infer"   # 保存先ディレクトリ

os.makedirs(save_dir, exist_ok=True) # 保存先ディレクトリ作成/exist_ok=True: もし save_dir（この場合は "runs/pose_infer"）が既に存在していてもエラーにせず、そのまま続行するというオプション

# 推論実行
# model.predict(...): model オブジェクトの predict メソッド（機能）を呼び出して推論を実行
results = model.predict(source=source, imgsz=imgsz, conf=conf)

# results は iterable(反復可能)。複数フレーム/画像に対して処理する。
# 複数の結果（例えば動画の各フレーム、またはフォルダ内の各画像）を保持できる
for i, r in enumerate(results):
    # 可視化画像を取得して保存 (r.plot() はRGB画像を返す)
    try:
        annotated = r.plot()  # numpy array (RGB)
        out_path = os.path.join(save_dir, f"annotated_{i}.jpg")
        # OpenCV はBGRで扱うため変換して保存
        cv2.imwrite(out_path, cv2.cvtColor(annotated, cv2.COLOR_RGB2BGR))
        print(f"Saved annotated image: {out_path}")
    except Exception as e:
        print("Could not plot/save annotated image:", e)

    # キーポイントがあれば取り出して表示 (モデルの出力形式に依存するため安全にアクセス)
    try:
        # ultralytics のバージョンにより属性名が異なることがあるので安全に取得
        if hasattr(r, "keypoints") and r.keypoints is not None:
            kps = r.keypoints
        elif hasattr(r, "masks") and r.masks is not None:
            kps = r.masks
        else:
            kps = None

        if kps is not None:
            # kps を表示しやすい形に変換して出力
            try:
                arr = np.array(kps)  # 多くの場合 numpy に変換可能
            except Exception:
                arr = kps
            print(f"Frame {i} keypoints / mask info:", type(arr), getattr(arr, "shape", None))
            # 必要ならここで arr を解析して各姿勢のキーポイント座標を処理
        else:
            print(f"Frame {i}: no keypoints found in result")
    except Exception as e:
        print("Error while extracting keypoints:", e)
# ...existing code...

Creating new Ultralytics Settings v0.0.6 file  
View Ultralytics Settings with 'yolo settings' or at 'C:\Users\haruk\AppData\Roaming\Ultralytics\settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.



FileNotFoundError: /2011tokyo_mister_fp-011-320x480.jpg does not exist